<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_12_Garden_Groups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
input = """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""

In [56]:
input = open("input.txt").read()

In [57]:
def count_non_matching_sides(grid_str):
    grid = [list(line) for line in grid_str.splitlines()]
    rows, cols = len(grid), len(grid[0])
    visited = set()
    total = 0
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    def bfs(start_r, start_c):
        char = grid[start_r][start_c]
        queue = [(start_r, start_c)]
        component = set([(start_r, start_c)])
        perimeter = 0

        while queue:
            r, c = queue.pop(0)

            # Count non-matching sides
            for dr, dc in directions:
                new_r, new_c = r + dr, c + dc
                if (new_r < 0 or new_r >= rows or
                    new_c < 0 or new_c >= cols or
                    grid[new_r][new_c] != char):
                    perimeter += 1
                elif (new_r, new_c) not in component:
                    queue.append((new_r, new_c))
                    component.add((new_r, new_c))

        visited.update(component)
        return {char: perimeter}, len(component)

    # Process each unvisited cell
    for r in range(rows):
        for c in range(cols):
            if (r, c) not in visited:
                counts, freq = bfs(r, c)
                # Multiply counts by frequency and add to total
                total += sum(count * freq for count in counts.values())

    return total

count_non_matching_sides(input)

1461806

In [59]:
def count_non_matching_sides2(grid_str):
    grid = [list(line) for line in grid_str.splitlines()]
    rows, cols = len(grid), len(grid[0])
    visited = set()
    total = 0
    directions = [(-1, 0, 'n'), (1, 0, 's'), (0, -1, 'w'), (0, 1, 'e')]

    def bfs(start_r, start_c):
        char = grid[start_r][start_c]
        queue = [(start_r, start_c)]
        component = set([(start_r, start_c)])
        sides = []

        while queue:
            r, c = queue.pop(0)

            for dr, dc, direction in directions:
                new_r, new_c = r + dr, c + dc
                if (new_r < 0 or new_r >= rows or
                    new_c < 0 or new_c >= cols or
                    grid[new_r][new_c] != char):
                    sides.append((r, c, direction))
                elif (new_r, new_c) not in component:
                    queue.append((new_r, new_c))
                    component.add((new_r, new_c))

        visited.update(component)
        return sides, len(component)

    def merge_sides(sides):
        # Group edges by direction
        edges = {
            'n': set(),  # (y, x1, x2)
            's': set(),  # (y, x1, x2)
            'w': set(),  # (x, y1, y2)
            'e': set(),  # (x, y1, y2)
        }

        for r, c, direction in sides:
            if direction == 'n':
                edges['n'].add((r, c, c+1))
            elif direction == 's':
                edges['s'].add((r+1, c, c+1))
            elif direction == 'w':
                edges['w'].add((c, r, r+1))
            else:  # 'e'
                edges['e'].add((c+1, r, r+1))

        def merge_ranges(edges):
            if not edges:
                return []
            edges = sorted(edges)
            merged = []
            current = list(edges[0])

            for edge in edges[1:]:
                if edge[0] == current[0]:  # Same row/column
                    if edge[1] == current[2]:  # Adjacent
                        current[2] = edge[2]
                    elif edge[1] > current[2]:  # Gap
                        merged.append(tuple(current))
                        current = list(edge)
                else:  # Different row/column
                    merged.append(tuple(current))
                    current = list(edge)

            merged.append(tuple(current))
            return merged

        merged_edges = []
        for direction, edge_set in edges.items():
            merged = merge_ranges(edge_set)
            # print(f"Merged {direction}:", merged)
            merged_edges.extend(merged)

        return len(merged_edges)

    # Process each unvisited cell
    for r in range(rows):
        for c in range(cols):
            if (r, c) not in visited:
                sides, component_size = bfs(r, c)
                # print("\nRaw sides from BFS:", sides)
                polygon_sides = merge_sides(sides)
                # print(f"Polygon sides count: {polygon_sides}")
                total += polygon_sides * component_size

    return total

count_non_matching_sides2(input)

887932